## 11. Alignment of Model Scales

## Numerical experiments - Week 04/2025

_Boyan Mihaylov, MSc Computational Science (UVA/VU)_

The goal of this study is to cross-verify and combine the functionalities of the high-resolution, medium-resolution and low-resolution models, in order to obtain a universal mathematical relationship between cell wall permeability, spore density and spore coverage due to clustering.

## Prerequisite libraries

In [1]:
using PyPlot
using Revise

Revise.includet("./conversions.jl")
Revise.includet("./diffusion.jl")
Revise.includet("./setup.jl")
Revise.includet("./plotting.jl")
using .Conversions
using .Diffusion
using .Setup
using .Plotting

## 1. Spore clusters in the medium-resolution model

The first experiment compares a high-resolution simulation of a 6-neighbour cluster with varying distances on a lattice of size $L=256\times 0.2\mu\textrm{m}$ with an absorbing boundary to a medium-resolution simulation of analogical clusters on a lattice of size $L=256\times 5\mu\textrm{m}$ with a periodic boundary. This aims to investigate the following questions:
- Does the absorbing boundary bias results compared to a free flow in a much larger space domain?
- Can the medium-resolution model be used for a continuation of the distance-versus-decay-exponent function for larger distances?

